In [1]:
import copy
import torch
import deepsnap
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as pyg_nn

from sklearn.metrics import f1_score
from deepsnap.hetero_gnn import forward_op
from deepsnap.hetero_graph import HeteroGraph
from torch_sparse import SparseTensor, matmul

import pickle
import networkx as nx
import wandb
import optuna
import argparse
from multiprocessing.pool import ThreadPool

import os
from tqdm import tqdm

In [2]:
import sys
import os

# Get the current working directory
current_working_directory = os.getcwd()

# Construct the relative path (adjust the relative path as needed)
relative_path = os.path.join(current_working_directory, '../../train/gnn_llm/')

# Normalize the path to resolve any '..'
normalized_path = os.path.normpath(relative_path)

# Add the path to sys.path
sys.path.append(normalized_path)

# Now you can import your module
from hetero_gnn import HeteroGNN

In [3]:
def load_hetero_graph(G):
    hete = HeteroGraph(G, netlib=nx, directed=True)
    hete["node_target"] = hete._get_node_attributes("node_target")

    for key in hete["node_target"]:
        node_target = hete["node_target"][key]
        node_target = np.array(node_target)
        hete["node_target"][key] = torch.tensor(node_target)

    for key in hete["node_feature"]:
        node_feature = hete["node_feature"][key]
        node_feature = np.array(node_feature)
        hete["node_feature"][key] = torch.tensor(node_feature)

    del hete.G

    return hete

In [4]:
def get_batches_from_pickle(folder_path):
    pickle_files = os.listdir(folder_path)

    batches = []
    cpu_batches = []

    for file_name in pickle_files:
        file_path = os.path.join(folder_path, file_name)
        print(file_path)
        with open(file_path, "rb") as f:
            G = pickle.load(f)

        hetero_graph = load_hetero_graph(G)
        hetero_graph_cpu = load_hetero_graph(G)
        del G

        batches.append(hetero_graph)
        cpu_batches.append(hetero_graph_cpu)

    return batches, cpu_batches

In [8]:
# Get the current working directory
current_working_directory = os.getcwd()

# Construct the relative path to the directory with pickle files
output_directory = os.path.join(current_working_directory, "../../data/graphs/batches/gnn_only/test_small")

# Call your function
test_batches, test_batches_cpu = get_batches_from_pickle(output_directory)

c:\Users\adria\Desktop\MLG-Predicting-Impacful-Events\analysis\preds\../../data/graphs/batches/gnn_only/test_small\batch_00008.pkl


In [22]:
# dummy args
dummy_args = {
    # "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "device": "cuda",
    "hidden_size": 173,
    "epochs": 30,
    "weight_decay": 0.00002203762357664057,
    "lr": 0.003873757421883433,
    "attn_size": 63,
    "num_layers": 6,
    "aggr": "attn",
}

In [23]:
model = HeteroGNN(
        test_batches_cpu[0],
        dummy_args,
        num_layers=dummy_args["num_layers"],
        aggr=dummy_args["aggr"],
    ).to(dummy_args["device"])


['event', 'concept', 'event_target']


In [24]:
model.load_state_dict(torch.load("./best_model.pkl"))


RuntimeError: Error(s) in loading state_dict for HeteroGNN:
	Unexpected key(s) in state_dict: "convs.0.modules.6.lin_dst.weight", "convs.0.modules.6.lin_dst.bias", "convs.0.modules.6.lin_src.weight", "convs.0.modules.6.lin_src.bias", "convs.0.modules.6.lin_update.weight", "convs.0.modules.6.lin_update.bias", "convs.1.modules.6.lin_dst.weight", "convs.1.modules.6.lin_dst.bias", "convs.1.modules.6.lin_src.weight", "convs.1.modules.6.lin_src.bias", "convs.1.modules.6.lin_update.weight", "convs.1.modules.6.lin_update.bias", "convs.2.modules.6.lin_dst.weight", "convs.2.modules.6.lin_dst.bias", "convs.2.modules.6.lin_src.weight", "convs.2.modules.6.lin_src.bias", "convs.2.modules.6.lin_update.weight", "convs.2.modules.6.lin_update.bias", "convs.3.modules.6.lin_dst.weight", "convs.3.modules.6.lin_dst.bias", "convs.3.modules.6.lin_src.weight", "convs.3.modules.6.lin_src.bias", "convs.3.modules.6.lin_update.weight", "convs.3.modules.6.lin_update.bias", "convs.4.modules.6.lin_dst.weight", "convs

In [ ]:
for train_batch, train_batch_cpu in tqdm(zip(train_batches, train_batches_cpu), total=len(train_batches)):
 